In [1]:
import pandas as pd
from torchvision.datasets import Caltech256


caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories
caltech256_targets = pd.read_csv("data/caltech256.csv")

In [2]:
from library.taxonomy import Taxonomy


synthetic_taxonomy, domains = Taxonomy.create_synthetic_taxonomy(
    num_atomic_concepts=257,
    num_domains=2,
    domain_class_count_mean=180,
    domain_class_count_variance=10,
    concept_cluster_size_mean=3,
    concept_cluster_size_variance=1,
    has_no_prediction_class=True,
)

domain_A_mapping = domains[0].to_mapping()
domain_B_mapping = domains[1].to_mapping()

In [3]:
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers
from library.models import ResNetModel
from library.datasets import Caltech256MappedDataModule


# Reduce the precision of matrix multiplication to speed up training
torch.set_float32_matmul_precision("medium")

tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs", name="caltech256_synthetic_A")
dataset = Caltech256MappedDataModule(mapping=domain_A_mapping)
model_name = "resnet50-caltech256-synthetic-A-min-val-loss"
trainer = Trainer(
    max_epochs=20,
    logger=tb_logger,
    callbacks=[
        # Save the model with the lowest validation loss
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
            filename=model_name,
            enable_version_counter=False,
        )
    ],
)

TRAIN = False

if TRAIN:
    model = ResNetModel(
        num_classes=len(set(domain_A_mapping.values())),
        architecture="resnet50",
        optim="sgd",
        optim_kwargs={
            "lr": 0.01,
            "momentum": 0.9,
            "weight_decay": 5e-4,
        },
    )
    trainer.fit(model, datamodule=dataset)

    # Test with the best model from the checkpoint
    results = trainer.test(datamodule=dataset, ckpt_path="best")
else:
    model = ResNetModel.load_from_checkpoint(f"checkpoints/{model_name}.ckpt")
    results = trainer.test(model, datamodule=dataset)

print(results)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 35/35 [00:09<00:00,  3.75it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.8384755253791809
        eval_loss            0.770880401134491
        hp_metric           0.8384755253791809
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'eval_loss': 0.770880401134491, 'eval_accuracy': 0.8384755253791809, 'hp_metric': 0.8384755253791809}]


In [4]:
tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs", name="caltech256_synthetic_B")
dataset = Caltech256MappedDataModule(mapping=domain_B_mapping)
model_name = "resnet50-caltech256-synthetic-B-min-val-loss"
trainer = Trainer(
    max_epochs=20,
    logger=tb_logger,
    callbacks=[
        # Save the model with the lowest validation loss
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
            filename=model_name,
            enable_version_counter=False,
        )
    ],
)

TRAIN = False

if TRAIN:
    model = ResNetModel(
        architecture="resnet50",
        optim="sgd",
        optim_kwargs={
            "lr": 0.01,
            "momentum": 0.9,
            "weight_decay": 5e-4,
        },
        num_classes=len(set(domain_B_mapping.values())),
    )
    trainer.fit(model, datamodule=dataset)

    # Test with the best model from the checkpoint
    results = trainer.test(datamodule=dataset, ckpt_path="best")
else:
    model = ResNetModel.load_from_checkpoint(f"checkpoints/{model_name}.ckpt")
    results = trainer.test(model, datamodule=dataset)

print(results)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 33/33 [00:08<00:00,  3.75it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.8132184147834778
        eval_loss           0.7469330430030823
        hp_metric           0.8132184147834778
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'eval_loss': 0.7469330430030823, 'eval_accuracy': 0.8132184147834778, 'hp_metric': 0.8132184147834778}]


In [5]:
import pandas as pd
from torchvision.datasets import Caltech256


caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories
caltech256_targets = pd.read_csv("data/caltech256.csv")

In [6]:
import torch
from library.datasets import Caltech256MappedDataModule
from lightning.pytorch import Trainer


torch.set_float32_matmul_precision("medium")


PREDICT = False

if PREDICT:
    dataset_domain_A = Caltech256MappedDataModule(mapping=domain_A_mapping)
    model_domain_A = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-caltech256-synthetic-A-min-val-loss.ckpt"
    )
    model_domain_A.eval()

    dataset_domain_B = Caltech256MappedDataModule(mapping=domain_B_mapping)
    model_domain_B = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-caltech256-synthetic-B-min-val-loss.ckpt"
    )
    model_domain_B.eval()

    trainer = Trainer(logger=False, enable_checkpointing=False)
    model_A_on_domain_B = trainer.predict(model_domain_A, datamodule=dataset_domain_B)
    model_B_on_domain_A = trainer.predict(model_domain_B, datamodule=dataset_domain_A)

    predictions_A_on_B = torch.cat(model_A_on_domain_B).argmax(dim=1)  # type: ignore
    predictions_B_on_A = torch.cat(model_B_on_domain_A).argmax(dim=1)  # type: ignore

    domain_A_targets = torch.cat(
        [label for _, label in dataset_domain_A.predict_dataloader()]
    )
    domain_A_targets = domain_A_targets.map_(
        torch.zeros_like(domain_A_targets), lambda x, _: domain_A_mapping[x]
    )
    domain_B_targets = torch.cat(
        [label for _, label in dataset_domain_B.predict_dataloader()]
    )
    domain_B_targets = domain_B_targets.map_(
        torch.zeros_like(domain_B_targets), lambda x, _: domain_B_mapping[x]
    )

    df = pd.DataFrame(
        {
            "domain_A": domain_A_targets,
            "predictions_B_on_A": predictions_B_on_A,
        }
    )
    df.to_csv("data/caltech256_domain_A_predictions.csv", index=False)
    df = pd.DataFrame(
        {
            "domain_B": domain_B_targets,
            "predictions_A_on_B": predictions_A_on_B,
        }
    )
    df.to_csv("data/caltech256_domain_B_predictions.csv", index=False)

df_A = pd.read_csv("data/caltech256_domain_A_predictions.csv")
df_B = pd.read_csv("data/caltech256_domain_B_predictions.csv")

In [7]:
from library.taxonomy import Taxonomy
import numpy as np

taxonomy = Taxonomy.from_cross_domain_predictions(
    cross_domain_predictions=[
        (0, 1, np.array(df_B["predictions_A_on_B"], dtype=np.intp)),
        (1, 0, np.array(df_A["predictions_B_on_A"], dtype=np.intp)),
    ],
    domain_targets=[
        (0, np.array(df_A["domain_A"], dtype=np.intp)),
        (1, np.array(df_B["domain_B"], dtype=np.intp)),
    ],
    domain_labels=synthetic_taxonomy.domain_labels,
)
taxonomy.visualize_graph("Synthetic Model Taxonomy").save_graph(
    "output/caltech256_synthetic_model_taxonomy.html"
)
taxonomy.build_universal_taxonomy()
taxonomy.visualize_graph("Synthetic Model Universal Taxonomy").save_graph(
    "output/caltech256_synthetic_model_universal_taxonomy.html"
)

synthetic_taxonomy.visualize_graph("Synthetic Taxonomy").save_graph(
    "output/caltech256_synthetic_taxonomy.html"
)
synthetic_taxonomy.build_universal_taxonomy()
synthetic_taxonomy.visualize_graph("Synthetic Universal Taxonomy").save_graph(
    "output/caltech256_synthetic_universal_taxonomy.html"
)

In [8]:
edr = taxonomy.edge_difference_ratio(synthetic_taxonomy, weighted=True)